In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv
import os
import wrds
import numpy as np

load_dotenv()

sys.path.append(str(Path().resolve().parent))  # Adds the parent directory of 'main' to sys.path
import pandas as pd

def get_credentials():
    fred_api_key = os.getenv("FRED_API_KEY")
    wrds_username = os.getenv("WRDS_USERNAME")
    wrds_password = os.getenv("WRDS_PASSWORD")

    return fred_api_key, wrds_username, wrds_password

fred_api_key, wrds_username, wrds_password = get_credentials()

In [ ]:
def get_compustat(
    wrds_username: str,
    wrds_password: str,
    START_DATE: str = "01/01/2000",
    END_DATE: str = "12/31/2024",
):
    """
    Retrieve data from Compustat.
    """
    query = f"""
        select gvkey, datadate, seq, ceq, at, lt, txditc, txdb, itcb,  pstkrv,
        pstkl, pstk, capx, oancf, sale, cogs, xint, xsga 
        from comp.funda
        where consol='C' and popsrc='D' and indfmt='INDL' and datafmt='STD'
        and datadate between '{START_DATE}' and '{END_DATE}'
    """

    conn = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    funda = conn.raw_sql(query, date_cols=["datadate", "fdatem", "rdq"])
    conn.close()
    return funda

def get_compustat_gic_codes(wrds_username: str, wrds_password: str):
    query = """
            select gvkey, gsector, ggroup, gind, gsubind, indfrom, indthru
            from comp.co_hgic 
            """

    conn = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    gic = conn.raw_sql(query, date_cols=["indfrom", "indthru"])
    conn.close()
    return gic

def get_crsp_monthly(
    wrds_username: str,
    wrds_password: str,
    CRSP_START_DATE="20000101",
    CRSP_END_DATE="20241231",
) -> pd.DataFrame:
    """
    Download CRSP monthly stock file data and return a DataFrame.
    By the default, stocks with SHRCD between 10 and 11 and EXCHCD between 1 and 3 are selected.

    Args:
        wrds_username (str): A WRDS username to use for the connection.
        wrds_password (str): A WRDS password to use for the connection.
        CRSP_START_DATE (str): The start date for the data.
        CRSP_END_DATE (str): The end date for the data.
    """

    conn = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)

    query = f"""
    select a.date, a.permno, a.shrout, a.cfacpr, a.cfacshr, a.altprc, a.prc, a.vol, a.ret,
    b.ticker, b.comnam, b.exchcd, b.shrcd, b.ncusip
    from crsp.msf as a
    left join crsp.dsenames as b
    on a.PERMNO=b.PERMNO
    and b.namedt<=a.date
    and a.date<=b.nameendt
    where b.SHRCD between 10 and  11
    and b.EXCHCD between 1 and  3
    and a.date >= '{CRSP_START_DATE}' and a.date <= '{CRSP_END_DATE}'  """

    df = conn.raw_sql(query)
    df["date"] = pd.to_datetime(df["date"])
    # push date to the end of the month
    df["date"] = df["date"] + pd.offsets.MonthEnd(0)
    df["prc"] = np.abs(df["prc"])
    df["permno"] = df["permno"].astype(int)

    conn.close()

    return df

In [3]:
conn = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
crsp_desc = conn.describe_table(library='crsp', table='dsenames')  # For CRSP monthly stock file
crsp_desc.to_csv(os.path.join(os.getenv("DATADIR"), 'crsp_dsenames_description.csv'))
crsp_desc = conn.describe_table(library='crsp', table='msf')  # For CRSP monthly stock file
crsp_desc.to_csv(os.path.join(os.getenv("DATADIR"), 'crsp_msf_description.csv'))
crsp_link_desc = conn.describe_table(library='crsp', table='ccmxpf_linktable')  # For CRSP link table
crsp_link_desc.to_csv(os.path.join(os.getenv("DATADIR"), 'crsp_link_table_description.csv'))
comp_desc = conn.describe_table(library='comp', table='funda')  # For Compustat fundamentals
comp_desc.to_csv(os.path.join(os.getenv("DATADIR"), 'compustat_funda_description.csv'))

Loading library list...
Done
Approximately 117859 rows in crsp.dsenames.
Approximately 5153763 rows in crsp.msf.
Approximately 89677 rows in crsp.ccmxpf_linktable.
Approximately 928693 rows in comp.funda.


## CRSP Data

In [41]:
crsp = get_crsp_monthly(
    wrds_username=wrds_username,
    wrds_password=wrds_password
)

Loading library list...
Done


In [42]:
crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip
11,2000-12-31,10032,41067.0,1.0,1.0,30.39063,30.39063,301527.0,-0.249614,PLXS,PLEXUS CORP,3,11,72913210
23,2001-12-31,10032,41812.0,1.0,1.0,26.56,26.56,215943.0,-0.119071,PLXS,PLEXUS CORP,3,11,72913210
35,2002-12-31,10032,42128.0,1.0,1.0,8.78,8.78,176197.0,-0.422748,PLXS,PLEXUS CORP,3,11,72913210
47,2003-12-31,10032,42679.0,1.0,1.0,17.17,17.17,56538.0,-0.042921,PLXS,PLEXUS CORP,3,11,72913210
59,2004-12-31,10032,43199.0,1.0,1.0,13.01,13.01,88901.0,-0.053818,PLXS,PLEXUS CORP,3,11,72913210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294917,2010-12-31,10051,33192.0,1.0,1.0,21.19,21.19,55488.0,0.104797,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20
294929,2011-12-31,10051,33658.0,1.0,1.0,18.69,18.69,46117.0,0.168125,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20
294941,2012-12-31,10051,34248.0,1.0,1.0,27.36,27.36,31200.0,0.047874,HGR,HANGER INC,1,11,41043F20
294953,2013-12-31,10051,34768.0,1.0,1.0,39.34,39.34,38658.0,0.012873,HGR,HANGER INC,1,11,41043F20


In [43]:
# Compute market cap (ME) and scale by millions
crsp = (crsp
  .assign(mktcap=lambda x: x["shrout"]*x["altprc"]/1000000)
  .assign(mktcap=lambda x: x["mktcap"].replace(0, np.nan))
)

In [44]:
crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap
11,2000-12-31,10032,41067.0,1.0,1.0,30.39063,30.39063,301527.0,-0.249614,PLXS,PLEXUS CORP,3,11,72913210,1.248052
23,2001-12-31,10032,41812.0,1.0,1.0,26.56,26.56,215943.0,-0.119071,PLXS,PLEXUS CORP,3,11,72913210,1.110527
35,2002-12-31,10032,42128.0,1.0,1.0,8.78,8.78,176197.0,-0.422748,PLXS,PLEXUS CORP,3,11,72913210,0.369884
47,2003-12-31,10032,42679.0,1.0,1.0,17.17,17.17,56538.0,-0.042921,PLXS,PLEXUS CORP,3,11,72913210,0.732798
59,2004-12-31,10032,43199.0,1.0,1.0,13.01,13.01,88901.0,-0.053818,PLXS,PLEXUS CORP,3,11,72913210,0.562019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294917,2010-12-31,10051,33192.0,1.0,1.0,21.19,21.19,55488.0,0.104797,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20,0.703338
294929,2011-12-31,10051,33658.0,1.0,1.0,18.69,18.69,46117.0,0.168125,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20,0.629068
294941,2012-12-31,10051,34248.0,1.0,1.0,27.36,27.36,31200.0,0.047874,HGR,HANGER INC,1,11,41043F20,0.937025
294953,2013-12-31,10051,34768.0,1.0,1.0,39.34,39.34,38658.0,0.012873,HGR,HANGER INC,1,11,41043F20,1.367773


In [45]:
# Create lagged market cap variable for value-weighted portfolios
mktcap_lag = (crsp
  .assign(
    date=lambda x: x["date"]+pd.DateOffset(months=1),
    mktcap_lag=lambda x: x["mktcap"]
  )
  .get(["permno", "date", "mktcap_lag"])
)

# merge to data
crsp = (crsp
  .merge(mktcap_lag, how="left", on=["permno", "date"])
)

crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap,mktcap_lag
11,2000-12-31,10032,41067.0,1.0,1.0,30.39063,30.39063,301527.0,-0.249614,PLXS,PLEXUS CORP,3,11,72913210,1.248052,<NA>
23,2001-12-31,10032,41812.0,1.0,1.0,26.56,26.56,215943.0,-0.119071,PLXS,PLEXUS CORP,3,11,72913210,1.110527,<NA>
35,2002-12-31,10032,42128.0,1.0,1.0,8.78,8.78,176197.0,-0.422748,PLXS,PLEXUS CORP,3,11,72913210,0.369884,<NA>
47,2003-12-31,10032,42679.0,1.0,1.0,17.17,17.17,56538.0,-0.042921,PLXS,PLEXUS CORP,3,11,72913210,0.732798,<NA>
59,2004-12-31,10032,43199.0,1.0,1.0,13.01,13.01,88901.0,-0.053818,PLXS,PLEXUS CORP,3,11,72913210,0.562019,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294917,2010-12-31,10051,33192.0,1.0,1.0,21.19,21.19,55488.0,0.104797,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20,0.703338,<NA>
1294929,2011-12-31,10051,33658.0,1.0,1.0,18.69,18.69,46117.0,0.168125,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20,0.629068,<NA>
1294941,2012-12-31,10051,34248.0,1.0,1.0,27.36,27.36,31200.0,0.047874,HGR,HANGER INC,1,11,41043F20,0.937025,<NA>
1294953,2013-12-31,10051,34768.0,1.0,1.0,39.34,39.34,38658.0,0.012873,HGR,HANGER INC,1,11,41043F20,1.367773,<NA>


In [46]:
# Download risk-free rate
from main.data.download import get_ff5_factors_monthly
ff_factors_monthly = get_ff5_factors_monthly()
rf = ff_factors_monthly.get(["date", "rf"])


In [47]:
# Calculate excess return
crsp = crsp.merge(rf, how="left", on="date")
crsp['ret_excess'] = crsp['ret'] - crsp['rf']
crsp['ret_excess'] = crsp['ret_excess'].clip(lower=-1)
crsp = crsp.drop(columns=['rf'])

crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap,mktcap_lag,ret_excess
11,2000-12-31,10032,41067.0,1.0,1.0,30.39063,30.39063,301527.0,-0.249614,PLXS,PLEXUS CORP,3,11,72913210,1.248052,<NA>,-0.254614
23,2001-12-31,10032,41812.0,1.0,1.0,26.56,26.56,215943.0,-0.119071,PLXS,PLEXUS CORP,3,11,72913210,1.110527,<NA>,-0.120571
35,2002-12-31,10032,42128.0,1.0,1.0,8.78,8.78,176197.0,-0.422748,PLXS,PLEXUS CORP,3,11,72913210,0.369884,<NA>,-0.423848
47,2003-12-31,10032,42679.0,1.0,1.0,17.17,17.17,56538.0,-0.042921,PLXS,PLEXUS CORP,3,11,72913210,0.732798,<NA>,-0.043721
59,2004-12-31,10032,43199.0,1.0,1.0,13.01,13.01,88901.0,-0.053818,PLXS,PLEXUS CORP,3,11,72913210,0.562019,<NA>,-0.055418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294917,2010-12-31,10051,33192.0,1.0,1.0,21.19,21.19,55488.0,0.104797,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20,0.703338,<NA>,0.104697
1294929,2011-12-31,10051,33658.0,1.0,1.0,18.69,18.69,46117.0,0.168125,HGR,HANGER ORTHOPEDIC GROUP INC,1,11,41043F20,0.629068,<NA>,0.168125
1294941,2012-12-31,10051,34248.0,1.0,1.0,27.36,27.36,31200.0,0.047874,HGR,HANGER INC,1,11,41043F20,0.937025,<NA>,0.047774
1294953,2013-12-31,10051,34768.0,1.0,1.0,39.34,39.34,38658.0,0.012873,HGR,HANGER INC,1,11,41043F20,1.367773,<NA>,0.012873


In [48]:
# drop rows with missing excess return and market cap
crsp = crsp.dropna(subset=['ret_excess', 'mktcap', 'mktcap_lag'])

In [52]:
crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap,mktcap_lag,ret_excess


## Preparing Compustat


From Kenneth French's website under variable definitions we find: 

**BE**: Book equity is constructed from Compustat data or collected from the Moody’s Industrial, Financial, and Utilities manuals. BE is the book value of stockholders’ equity, plus balance sheet deferred taxes and investment tax credit (if available), minus the book value of preferred stock. Depending on availability, we use the redemption, liquidation, or par value (in that order) to estimate the book value of preferred stock. Stockholders’ equity is the value reported by Moody’s or Compustat, if it is available. If not, we measure stockholders’ equity as the book value of common equity plus the par value of preferred stock, or the book value of assets minus total liabilities (in that order)

**OP**: The operating profitability ratio used to form portfolios in June of year t is annual revenues minus cost of goods sold, interest expense, and selling, general, and administrative expense divided by the sum of book equity and minority interest for the last fiscal year ending in t-1.

In [10]:
comp = get_compustat(wrds_username=wrds_username, wrds_password=wrds_password)

Loading library list...
Done


In [11]:
comp['be'] = (
    comp['seq'].combine_first(comp['ceq'] + comp['pstk']).combine_first(comp['at'] - comp['lt'])
    + comp['txditc'].combine_first(comp['txdb'] + comp['itcb']).fillna(0)
    - comp['pstk'].combine_first(comp['pstkrv']).combine_first(comp['pstkl']).fillna(0)
)

# Note to self. Try replacing sale with reserves
comp['be'] = comp['be'].apply(lambda x: np.nan if x <= 0 else x)
comp['op'] =  ((comp["sale"]-comp["cogs"].fillna(0)- 
        comp["xsga"].fillna(0)-comp["xint"].fillna(0))/comp["be"])

**INV**: The investment ratio used to form portfolios in June of year t is the change in total assets from the fiscal year ending in year t-2 to the fiscal year ending in t-1, divided by t-2 total assets

In [12]:
comp['year'] = pd.DatetimeIndex(comp['datadate']).year

# Keep only the last observation for each firm-year combination
comp = comp.sort_values('datadate').groupby(['gvkey', 'year']).tail(1).reset_index()

# Calculate inv
compustat_lag = comp[['gvkey', 'year', 'at']].copy()
compustat_lag['year'] += 1
compustat_lag = compustat_lag.rename(columns={'at': 'at_lag'})
comp = comp.merge(compustat_lag, on=['gvkey', 'year'], how='left')

In [13]:
comp.columns

Index(['index', 'gvkey', 'datadate', 'seq', 'ceq', 'at', 'lt', 'txditc',
       'txdb', 'itcb', 'pstkrv', 'pstkl', 'pstk', 'capx', 'oancf', 'sale',
       'cogs', 'xint', 'xsga', 'be', 'op', 'year', 'at_lag'],
      dtype='object')

In [14]:
comp = (comp
    .assign(
        inv = lambda x: (x['at'] - x['at_lag']) / x['at_lag']
    )
    .assign(
        inv = lambda x: x['inv'].where(x['at_lag'] > 0)
    )
)


## Linking compustat and CRSP

In [15]:
def get_crsp_compu_link_table(
    wrds_username: str, wrds_password: str
) -> pd.DataFrame:
    """
    Download CRSP-Compustat link table and return a DataFrame.

    Args:
        wrds_username (str): A WRDS username to use for the connection.
        wrds_password (str): A WRDS password to use for the connection.
    """
    import wrds

    conn = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)

    query = """
    SELECT lpermno AS permno, gvkey, linkdt, COALESCE(linkenddt, CURRENT_DATE) AS linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE usedflag = 1
    AND linkprim IN ('P', 'C')
    """

    # Execute the query
    data = conn.raw_sql(query)
    data["linkdt"] = pd.to_datetime(data["linkdt"])
    data["linkenddt"] = pd.to_datetime(data["linkenddt"])

    conn.close()
    return data


In [16]:
link_table = get_crsp_compu_link_table(
    wrds_username=wrds_username, wrds_password=wrds_password
)


Loading library list...
Done


In [37]:
crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap,mktcap_lag,ret_excess


In [29]:
link_table[link_table.linkenddt.dt.month == 12]

,permno,gvkey,linkdt,linkenddt
13,10103.0,001012,1978-01-31,1989-12-29
14,50906.0,001013,1979-03-16,2010-12-31
36,65832.0,001034,1984-02-15,2008-12-29
39,66413.0,001038,1983-08-18,2004-12-31
56,41961.0,001051,1968-01-02,1968-12-31
...,...,...,...,...
38688,14344.0,315887,2013-12-12,2023-12-29
38700,16496.0,318728,2016-12-09,2021-12-31
38702,16469.0,319507,2016-11-18,2019-12-31
38727,19989.0,325942,2020-10-05,2020-12-31


In [18]:
ccm_links = crsp.merge(link_table, how="inner", on="permno")
# ccm_links["linkenddt"] = ccm_links["linkenddt"].fillna(pd.to_datetime('2024-12-31'))
ccm_links = ccm_links.query("~gvkey.isnull() & (date >= linkdt) & (date <= linkenddt)")
ccm_links = ccm_links.get(['permno', 'gvkey', 'date'])

In [19]:
ccm_links[ccm_links.date.dt.month == 12]

,permno,gvkey,date


In [20]:
crsp = crsp.merge(ccm_links, how='left', on=['permno', 'date'])

In [21]:
crsp[crsp.date.dt.month == 12]

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap,mktcap_lag,ret_excess,gvkey


## Replication

In [22]:
size = crsp.query("date.dt.month == 6").copy()
size['sorting_date'] = size['date'] + pd.DateOffset(months=1)
size = size[['permno', 'exchcd', 'sorting_date', 'mktcap']].rename(columns={'mktcap': 'size'})

In [23]:
size

,permno,exchcd,sorting_date,size
2,10000,3,1986-07-30,-0.011735
11,10001,3,1986-07-30,-0.006033
18,10001,3,1987-07-30,0.005822
25,10001,3,1988-07-30,0.0062
32,10001,3,1989-07-30,0.007007
...,...,...,...,...
1963719,93436,3,2020-07-30,200.844671
1963726,93436,3,2021-07-30,668.826849
1963733,93436,3,2022-07-30,701.030199
1963740,93436,3,2023-07-30,830.857948


In [24]:
crsp.date[crsp.date.dt.month == 12]

Series([], Name: date, dtype: datetime64[ns])

In [25]:
me = crsp.query("date.dt.month == 12").copy()
# me['sorting_date'] = me['date'] + pd.DateOffset(months=7)
# me = me[['permno', 'gvkey', 'sorting_date', 'mktcap']].rename(columns={'mktcap': 'me'})

In [26]:
me

,date,permno,shrout,cfacpr,cfacshr,altprc,prc,vol,ret,ticker,comnam,exchcd,shrcd,ncusip,mktcap,mktcap_lag,ret_excess,gvkey


In [27]:
me.sorting_date

AttributeError: 'DataFrame' object has no attribute 'sorting_date'